<a href="https://colab.research.google.com/github/ivans14/Deep_Learning_Proj/blob/main/CNN_using_VGGM_for_ball_and_player_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ball and players detection for SoccerNet
### Using CNN and the pretained model VGGM

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import torch.optim as optim
import torch
from torch import nn
from torch.nn import LocalResponseNorm

from sklearn import metrics

In [ ]:
# Import dataset as: train, validation and test splits


In [ ]:
# Check some info regarding the images


In [7]:
# CNN model
out_features = 2 # Ball and players
image_shape = (400, 400, 3)

class Model(nn.Module):
    def __init__(self,out_classes, image_size):
        super().__init__()
        self.out_classes = out_classes
        H = image_size[0]
        W = image_size[1]
        C = image_size[2]

        conv1 = {
            "in": 1,
            "out": 16,
            "kernel": 5,
            "stride": 2,
            "padding": 0
        }
        conv2 = {
            "in":conv1["out"],
            "out": 28,
            "kernel": 5,
            "stride": 2,
            "padding": 0
        }
        pool = {
            "kernel": 2,
            "stride": 2,
            "padding": 0
        }

        conv_param = [conv1,pool,conv2,pool]

        for i in range (0, len(conv_param)):
            H = np.floor((H + 2*conv_param[i]["padding"] - conv_param[i]["kernel"])/conv_param[i]["stride"] + 1)
            W = np.floor((W + 2*conv_param[i]["padding"] - conv_param[i]["kernel"])/conv_param[i]["stride"] + 1)


        activation_fn = nn.ReLU(0.1)
        # Defining a Sequential pipeline for the entire CNN
        self.net = nn.Sequential(
            nn.Conv2d(in_channels = conv_param[0]["in"],
                     out_channels = conv_param[0]["out"],
                     kernel_size = conv_param[0]["kernel"],
                     stride= conv_param[0]["stride"],
                     padding= conv_param[0]["padding"]),
            activation_fn,
            LocalResponseNorm(2),   # Check: https://towardsdatascience.com/difference-between-local-response-normalization-and-batch-normalization-272308c034ac
            nn.MaxPool2d(kernel_size = conv_param[1]["kernel"],
                        stride =conv_param[1]["stride"]),
            
            nn.Conv2d(in_channels = conv_param[2]["in"],
                     out_channels = conv_param[2]["out"],
                     kernel_size = conv_param[2]["kernel"],
                     stride= conv_param[2]["stride"],
                     padding= conv_param[2]["padding"]),
            nn.MaxPool2d(kernel_size = conv_param[3]["kernel"],
                        stride =conv_param[3]["stride"]),
            
            #nn.Flatten(), # Flattens a contiguous range of dims into a tensor.
            # FFNN
            activation_fn,
            nn.Linear(int(conv_param[-2]["out"] * H * W), out_features),
            # nn.Dropout(0.5),
            # nn.Sigmoid(),
            # nn.Linear(,self.out_classes),
            # nn.Dropout(0.25),
            # nn.Sigmoid()
        )


    def forward(self, x):

      return nn.Softmax(self.net(x))

NN = Model(out_features, image_shape)
device = device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')  # use cuda if possible
NN.to(device)
print(NN)

# Definition of optimizer and loss functions
optimizer = optim.Adam(NN.parameters(),lr=0.008)
loss_fn = nn.CrossEntropyLoss() 

Model(
  (net): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 28, kernel_size=(5, 5), stride=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): ReLU(inplace=True)
    (7): Linear(in_features=16128, out_features=2, bias=True)
  )
)


In [ ]:
## VGGM pre-trained model


In [ ]:
# Define training function
# HOLA 
# adios

def accuracy(target, pred):
    return metrics.accuracy_score(target.detach().cpu().numpy(), pred.detach().cpu().numpy())

# Set number of epochs according to computational power, time and results.
batch_size = 64
num_epochs = 4
validation_every_steps = 500

step = 0
NN.train()

train_accuracies = []
valid_accuracies = []
running_loss = 0



for epoch in range(num_epochs):
    
    train_accuracies_batches = []
    train_loss_batches = []
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Forward pass, compute gradients, perform one training step.
        # Your code here!
        output = NN(inputs) # Pass the inouts through the NN
        loss = loss_fn(output,targets) # Compute and Save loss
        optimizer.zero_grad() #Clean up gradients
        loss.backward() # Compute gradients based on the loss from the current batch
        optimizer.step()
        running_loss += loss.item()

        # Increment step counter
        step += 1
        
        # Compute accuracy.
        predictions = output.max(1)[1]
        train_accuracies_batches.append(accuracy(targets, predictions))
        
        if step % validation_every_steps == 0:
            
            validation_loss = 0
            # Append average training accuracy to list.
            train_accuracies.append(np.mean(train_accuracies_batches))
            
            train_accuracies_batches = []
        
            # Compute accuracies on validation set.
            valid_accuracies_batches = []
            with torch.no_grad():
                NN.eval()
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    output = NN(inputs)
                    loss = loss_fn(output, targets)

                    predictions = output.max(1)[1]

                    # Multiply by len(x) because the final batch of DataLoader may be smaller (drop_last=False).
                    valid_accuracies_batches.append(accuracy(targets, predictions) * len(inputs))

                NN.train()
                
            # Append average validation accuracy to list.
            valid_accuracies.append(np.sum(valid_accuracies_batches) / len(test_set))
     
            print(f"Step {step:<5}   training accuracy: {train_accuracies[-1]}")
            print(f"             test accuracy: {valid_accuracies[-1]}")

# Save model (i.e. its weights)
torch.save(NN.state_dict(), 'trained_models/CNN_weights.pth')
print("Finished training.")